#<center>Trading Inteligente - Utilizando Análise de Séries <br> Temporais para Prever Valores  Futuros de Ações da B3</center>

---

Trabalho de análise preditiva de dados, que visa possibilitar a previsão de valores/tendências futuras em cotações de ativos listados na B3 e, assim, servir como ferramenta de orientação para investidores de curto prazo em seu processo de tomada de decisões. Para tal análise, será utilizada série temporal contendo os dados históricos dos ativos da B3, na qual será aplicado o método de análise estatística ARIMA.



---


*Projeto de análise de dados criado por **Dimas Tadeu Parreiras**, em caráter de aprovação da disciplima de Trabalho de Conclusão de Curso na Pós-Graduação em Ciência de Dados e Big Data pela Pontifícia Universidade Católica de Minas Gerais (PUC Minas).*


### ***Bibliotecas***

In [1]:
pip install pmdarima

In [2]:
import os
import pandas as pd
import pandas_datareader.data as wb
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from plotly import io
io.renderers.default = 'colab'
import warnings
warnings.filterwarnings("ignore")
import pmdarima as pm
from pmdarima.arima import ndiffs
from pmdarima.metrics import smape
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.stattools import adfuller

### ***Explorando o Dataset***

#### ***Escolhendo Uma Ação para Análise***

In [3]:
#codigo = input('Informe o código da ação desejada: ')
codigo = 'IBOVESPA'

#### ***Carregando os Dados***

In [4]:
#Carregando os dados do Yahoo Finance
ds_acao = pd.DataFrame()
ds_acao = wb.DataReader(name = '^BVSP', 
                        data_source = 'yahoo', 
                        start = '2000-1-1',
                        end = None)

#### ***Descrição dos Dados***

In [5]:
ds_acao.describe()

,High,Low,Open,Close,Volume,Adj Close
count,5190.000000,5190.000000,5190.000000,5190.000000,5.190000e+03,5190.000000
mean,50505.945239,49453.192477,49975.383153,49994.443681,7.254955e+06,49994.443681
std,26171.313437,25738.744515,25951.312241,25968.361630,2.372831e+07,25968.361630
min,8513.000000,8225.000000,8397.000000,8371.000000,0.000000e+00,8371.000000
25%,25617.750000,25035.000000,25312.500000,25325.250000,0.000000e+00,25325.250000
50%,53310.500000,52185.500000,52746.000000,52759.000000,2.351400e+06,52759.000000
75%,65514.000000,64242.750000,64857.750000,64862.750000,3.925050e+06,64862.750000
max,119593.000000,118108.000000,119528.000000,119528.000000,2.322653e+08,119528.000000


#### ***Verificando Existência de Dados Ausentes***

In [6]:
ds_acao.dropna(inplace=True)

#### ***Verificando Existência de Dados Duplicados***

In [7]:
ds_acao.drop_duplicates(inplace=True)

#### ***Convertendo as Data para DateTime***

In [8]:
#Convertendo e ordenando o conteúdo pela data
ds_acao.index = pd.to_datetime(ds_acao.index)
ds_acao.sort_index

<bound method DataFrame.sort_index of                      High            Low  ...      Volume      Adj Close
Date                                      ...                           
2000-01-03   17408.000000   16719.000000  ...         0.0   16930.000000
2000-01-04   16908.000000   15851.000000  ...         0.0   15851.000000
2000-01-05   16302.000000   15350.000000  ...         0.0   16245.000000
2000-01-06   16499.000000   15977.000000  ...         0.0   16107.000000
2000-01-07   16449.000000   16125.000000  ...         0.0   16309.000000
...                   ...            ...  ...         ...            ...
2020-12-18  119370.000000  117639.000000  ...   8636000.0  117679.000000
2020-12-21  118021.000000  114730.000000  ...  10347100.0  116016.000000
2020-12-22  116903.000000  115648.000000  ...   6947900.0  116348.000000
2020-12-23  118311.000000  116636.000000  ...   6483300.0  117857.000000
2020-12-28  119212.789062  117804.953125  ...         0.0  119123.703125

[5190 rows x

### ***Análise Exploratória***

#### ***Análise Gráfica***

In [9]:
#Representação Gráfica dos Valores de Fechamento ao Longo do Tempo
titulo = codigo +  ' | Cotações Históricas'
trace = [go.Scatter(x = ds_acao.index, 
                    y = ds_acao.Close)]
layout = go.Layout(
    xaxis = dict(title = 'Ano'),
    yaxis = dict(title = 'Valor da Ação'),
    title = titulo)
fig = go.Figure(data = trace, layout = layout)
fig.show()

In [10]:
#Representação Gráfica dos Valores de Negociação ao Longo do Últimos 60 dias de Operação da Bolsa
data = [go.Candlestick(x = ds_acao.tail(60).index,
                      open = ds_acao.Open.tail(60),
                      high = ds_acao.High.tail(60),
                      low = ds_acao.Low.tail(60),
                      close = ds_acao.Close.tail(60))]
fig = go.Figure(data=data)
title = codigo + ' - Valores de Negociação dos Últimos 60 dias de Operação na B3'
fig.layout.title = title
fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

#### ***Correlação Entre os Valores***

In [11]:
#Correlação entre as variáveis (Correlação de Pearson)
ds_acao.corr()

,High,Low,Open,Close,Volume,Adj Close
High,1.000000,0.999599,0.999737,0.999708,0.035142,0.999708
Low,0.999599,1.000000,0.999599,0.999765,0.034827,0.999765
Open,0.999737,0.999599,1.000000,0.999349,0.034980,0.999349
Close,0.999708,0.999765,0.999349,1.000000,0.035428,1.000000
Volume,0.035142,0.034827,0.034980,0.035428,1.000000,0.035428
Adj Close,0.999708,0.999765,0.999349,1.000000,0.035428,1.000000


In [12]:
# Representação gráfica das correlações
fig = make_subplots(rows=3, cols=2, start_cell="top-left")

# Definindo Correlações
fig.add_trace(go.Scatter(x = ds_acao['Open'], 
                         y = ds_acao['Close'], 
                         mode = 'markers', 
                         name = "Fechamento x Abertura"), 
                         row=1, col=1)
fig.add_trace(go.Scatter(x = ds_acao['Low'], 
                         y = ds_acao['Close'], 
                         mode = 'markers', 
                         name = "Fechamento x Mínima"), 
              row=1, col=2)
fig.add_trace(go.Scatter(x = ds_acao['High'], 
                         y = ds_acao['Close'], 
                         mode = 'markers', 
                         name = "Fechamento x Máxima"), 
                         row=2, col=1)
fig.add_trace(go.Scatter(x = ds_acao['Adj Close'], 
                         y = ds_acao['Close'], 
                         mode = 'markers', 
                         name = "Fechamento x Fechamento Ajustado"), 
                         row=2, col=2)
fig.add_trace(go.Scatter(x = ds_acao['Volume'], 
                         y = ds_acao['Close'], 
                         mode = 'markers', 
                         name = "Fechamento x Volume"), 
                         row=3, col=1)

# Propriedade do Eixo X
fig.update_xaxes(title_text="Abertura", row=1, col=1)
fig.update_xaxes(title_text="Mínima", row=1, col=2)
fig.update_xaxes(title_text="Máxima", row=2, col=1)
fig.update_xaxes(title_text="Fechamento Ajustado", row=2, col=2)
fig.update_xaxes(title_text="Volume", row=3, col=1)

# Propriedade do Eixo Y
fig.update_yaxes(title_text="Fechamento", row=1, col=1)
fig.update_yaxes(title_text="Fechamento", row=1, col=2)
fig.update_yaxes(title_text="Fechamento", row=2, col=1)
fig.update_yaxes(title_text="Fechamento", row=2, col=2)
fig.update_yaxes(title_text="Fechamento", row=3, col=1)

# Atualizando Layout
fig.update_layout(title_text= codigo + " | Correlação entre Valores", height=900, legend = dict( bgcolor="LightSteelBlue"), legend_orientation="h")

fig.show()

#### ***Retornos Diários***

In [13]:
log_retorno=np.log(ds_acao.Close/ds_acao.Close.shift(1))

In [14]:
#Representação Gráfica dos Retornos Obtidos ao Longo do Tempo
titulo = codigo +  ' | Retornos Diários'
trace = [go.Scatter(x = log_retorno.index, 
                    y = log_retorno)]
layout = go.Layout(xaxis = dict(title = 'Ano'),
                   yaxis = dict(title = 'Log Retorno'),
                   title = titulo)
fig = go.Figure(data = trace, layout = layout)
fig.show()

#### ***Retornos diários médios x Retornos anuais médios***

In [15]:
#Média Diária dos Retornos
media_diaria = log_retorno.mean()
print("A média diária dos retornos da ação", codigo, "é:", media_diaria)

A média diária dos retornos da ação IBOVESPA é: 0.00037600217365511764


In [16]:
#Média Anual dos Retornos (250 = número aproximado de dias úteis em um ano)
media_anual = log_retorno.mean()*250
print("A média anual dos retornos da ação", codigo, "é:", media_anual)

A média anual dos retornos da ação IBOVESPA é: 0.0940005434137794


#### ***Risco (desvio padrão)***

In [17]:
desvio = log_retorno.std()
print("O desvio padrão dos retornos da ação", codigo, "é:", media_anual)

O desvio padrão dos retornos da ação IBOVESPA é: 0.0940005434137794


#### ***Estacionaridade***

In [18]:
def estacionariedade(timeseries):
    
    #Valores Estatísticos
    media_variavel = [np.mean(timeseries[:x]) for x in range(len(timeseries))]
    variancia = [np.std(timeseries[:x]) for x in range(len(timeseries))]

    #Plotando os Gráficos
    trace1 = go.Scatter(x = timeseries.index, 
                        y = timeseries, 
                        line={'color': '#44bd32'}, 
                        name='Valores de Fechamento')
    trace2 = go.Scatter(x = timeseries.index, 
                        y = media_variavel, 
                        line={'color': '#c23616'}, 
                        name='Média Variável')
    trace3 = go.Scatter(x = timeseries.index, 
                        y = variancia, 
                        line = {'color': '#273c75'}, 
                        name ='Desvio Padrão Móvel')
    layout = go.Layout(xaxis = dict(title = 'Ano'),
                      yaxis = dict(title = 'Valores'),
                      title = titulo,
                      legend = dict( bgcolor = "LightSteelBlue"), 
                      legend_orientation = "h")
    data = [trace1, trace2, trace3]
    fig=go.Figure(data = data, layout = layout)
    fig.show()
    
    #Teste de Dickey-Fuller
    print ('Resultado do Teste de Dickey-Fuller:')
    dftest = adfuller(timeseries, 
                      autolag = 'AIC')
    dfoutput = pd.Series(dftest[0:4], 
                         index = ['Teste Estatístico','p-value','#Lags','Número de Observações'])
    for key,value in dftest[4].items():
        dfoutput['Valores Críticos(%s)'%key] = value
    print (dfoutput)

In [19]:
estacionariedade(ds_acao.Close)

Resultado do Teste de Dickey-Fuller:
Teste Estatístico          -0.482186
p-value                     0.895446
#Lags                      32.000000
Número de Observações    5157.000000
Valores Críticos(1%)       -3.431619
Valores Críticos(5%)       -2.862101
Valores Críticos(10%)      -2.567068
dtype: float64


### ***Análise Preditiva***

#### ***Dividindo os Dados***

In [20]:
i_teste = '2019-12-30' #Índice de ínicio do intervalo de teste
i_validacao = '2020-10-01' #Índice de ínicio do intervalo de validação

In [21]:
#Representação gráfica dos intervalos
titulo = codigo +  ' | Bases de Treino, Teste e Validação'
trace1  = go.Scatter(x = ds_acao[:i_teste].index, 
                     y = ds_acao[:i_teste].Close, 
                     line = {'color': '#2196F3'}, 
                     name = 'Intervalo de Treino')
trace2  = go.Scatter(x = ds_acao[i_teste:i_validacao].index, 
                     y = ds_acao[i_teste:i_validacao].Close, 
                     line = {'color': '#9C27B0'}, 
                     name ='Intervalo de Teste')
trace3  = go.Scatter(x = ds_acao[i_validacao:].index, 
                     y =ds_acao[i_validacao:].Close, 
                     line = {'color': '#F44336'}, 
                     name = 'Intervalo de Validação')
layout = go.Layout(xaxis=dict(title='Ano'),
                   yaxis=dict(title='Valor da Ação'),
                   legend = dict(bgcolor="LightSteelBlue"), 
                   legend_orientation="h",
                   title = titulo)
data = [trace1, trace2, trace3]
fig=go.Figure(data=data, layout=layout)
fig.show()

#### ***Auto Arima***

In [22]:
def def_auto(ts_treino):
  auto = pm.auto_arima(ts_treino, seasonal=False, stepwise=True,
                      suppress_warnings=True, error_action="ignore",
                      max_order=None, trace=True)
  return auto

In [23]:
def prever_um_passo(modelo):
    prev = modelo.predict(n_periods=1)
    return (prev.tolist()[0])

In [24]:
def aplicar_arima(ts_treino, ts_teste):
  modelo = def_auto(ts_treino)
  print('Termos p, d, q:', modelo.order)
  previsoes = []
  for nova_obs in ts_teste:
    prev = prever_um_passo(modelo)
    previsoes.append(prev)
    modelo.update(nova_obs)
  print(f"Mean squared error: {mean_squared_error(ts_teste, previsoes)}")
  print(f"SMAPE: {smape(ts_teste, previsoes)}")
  return previsoes

#### ***Teste do Modelo***

In [25]:
previsoes = aplicar_arima(ds_acao[:i_teste].Close.values, ds_acao[i_teste:i_validacao].Close.values)

Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=80165.563, Time=7.05 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=80176.846, Time=0.12 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=80176.745, Time=0.28 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=80176.617, Time=0.31 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=80177.917, Time=0.05 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=80169.812, Time=1.98 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=80173.858, Time=1.24 sec
 ARIMA(3,1,2)(0,0,0)[0] intercept   : AIC=80174.426, Time=2.97 sec
 ARIMA(2,1,3)(0,0,0)[0] intercept   : AIC=80174.071, Time=3.26 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=80169.458, Time=2.33 sec
 ARIMA(1,1,3)(0,0,0)[0] intercept   : AIC=80172.099, Time=2.64 sec
 ARIMA(3,1,1)(0,0,0)[0] intercept   : AIC=80172.543, Time=1.46 sec
 ARIMA(3,1,3)(0,0,0)[0] intercept   : AIC=80169.941, Time=9.51 sec
 ARIMA(2,1,2)(0,0,0)[0]             : AIC=80174.270, Time=0.59 sec

Best model:  ARIMA

##### ***Resultados Obtidos no Treino***

In [26]:
#Visualição Gráfica dos dados originais e do resultado obtido
titulo = codigo +  ' | Bases de Treino, Teste e Validação'
trace1  = go.Scatter(x = ds_acao[:i_teste].index, 
                     y = ds_acao[:i_teste].Close, 
                     line={'color': '#44bd32'}, 
                     name='Treino')
trace2  = go.Scatter(x = ds_acao[i_teste:i_validacao].index, 
                     y = ds_acao[i_teste:i_validacao].Close, 
                     line={'color': '#273c75'}, 
                     name='Teste')
trace3  = go.Scatter(x = ds_acao[i_teste:i_validacao].index,
                     y = previsoes, 
                     line={'color': '#FF5722'}, 
                     name='Validação')
layout = go.Layout(xaxis=dict(title = 'Ano'),
                    yaxis=dict(title = 'Valor da Ação'),
                    legend = dict(bgcolor="LightSteelBlue"), 
                    legend_orientation="h",
                    title = titulo)
data = [trace1, trace2, trace3]
fig=go.Figure(data=data, layout=layout)
fig.show()

In [27]:
#Visualização gráfica detalhada comparando apenas o período previsto no teste
data = [trace2, trace3]
fig=go.Figure(data=data, layout=layout)
fig.show()

#### ***Validação do Modelo***

In [28]:
previsoes = aplicar_arima(ds_acao[:i_validacao].Close.values, ds_acao[i_validacao:].Close.values)

Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=84840.796, Time=1.99 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=84876.358, Time=0.14 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=84836.667, Time=0.30 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=84838.487, Time=0.32 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=84875.707, Time=0.05 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=84836.779, Time=0.37 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=84837.037, Time=0.73 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=84838.786, Time=0.44 sec
 ARIMA(1,1,0)(0,0,0)[0]             : AIC=84836.305, Time=0.11 sec
 ARIMA(2,1,0)(0,0,0)[0]             : AIC=84836.352, Time=0.20 sec
 ARIMA(1,1,1)(0,0,0)[0]             : AIC=84836.599, Time=0.34 sec
 ARIMA(0,1,1)(0,0,0)[0]             : AIC=84838.123, Time=0.17 sec
 ARIMA(2,1,1)(0,0,0)[0]             : AIC=84838.354, Time=0.22 sec

Best model:  ARIMA(1,1,0)(0,0,0)[0]          
Total fit time: 5.378 seconds
Termos p,

##### ***Resultados Obtidos na Validação***

In [29]:
#Visualição Gráfica dos dados originais e do resultado obtido
titulo = codigo +  ' | Bases de Treino, Teste e Validação'
trace1  = go.Scatter(x = ds_acao[:i_validacao].index, 
                     y = ds_acao[:i_validacao].Close, 
                     line={'color': '#44bd32'}, 
                     name='Treino')
trace2  = go.Scatter(x = ds_acao[i_validacao:].index, 
                     y = ds_acao[i_validacao:].Close, 
                     line={'color': '#273c75'}, 
                     name='Teste')
trace3  = go.Scatter(x = ds_acao[i_validacao:].index,
                     y = previsoes, 
                     line={'color': '#FF5722'}, 
                     name='Validação')
layout = go.Layout(xaxis=dict(title = 'Ano'),
                    yaxis=dict(title = 'Valor da Ação'),
                    legend = dict(bgcolor="LightSteelBlue"), 
                    legend_orientation="h",
                    title = titulo)
data = [trace1, trace2, trace3]
fig=go.Figure(data=data, layout=layout)
fig.show()

In [30]:
#Visualização gráfica detalhada comparando apenas o período previsto no teste
data = [trace2, trace3]
fig=go.Figure(data=data, layout=layout)
fig.show()